In [ ]:
import pandas as pd
from pandas import DataFrame
from datetime import datetime
import os
import sys
import numpy as np
import seaborn as sns
import statistics
import random
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import pearsonr

os.getcwd()
workdir = "YOUR WORKING Directory"
datadir = workdir + "Data/"
rawdir = datadir + "raw_data/"
resultdir = datadir + "input_result/"
predictiondir = datadir + "prediction_result/"
modeldir = workdir + "model/"

today = datetime.today().strftime("%Y%m%d")
today

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
StartTime1 = datetime.now()
print("StartTime :", StartTime1)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
    


EndTime1 = datetime.now()
print("EndTime :", EndTime1)

In [ ]:
# load whole dataset from npz
train_data = np.load(resultdir + 'toy_train.npz')
val_data = np.load(resultdir + 'toy_val.npz')
test_data = np.load(resultdir + 'toy_test.npz')

In [ ]:
train_data.files, val_data.files, test_data.files

In [ ]:
x_train, y_train = train_data['x'], train_data['y']
x_val, y_val = val_data['x'], val_data['y']
x_test, y_test = test_data['x'], test_data['y']

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_val.shape, y_val.shape

# Random forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 200
                           ,random_state = 15
                           , max_depth=30
                           , max_features='sqrt'
                           , bootstrap=False
                           , min_samples_leaf=1
                           , min_samples_split=2)

In [ ]:
# training model

with K.tf.device('/GPU:0'):
    
    print("step : model training")
    StartTime = datetime.now()
    print("StartTime :", StartTime)
    
    rf.fit(x_train,y_train)
    
    EndTime = datetime.now()
    print("EndTime :", EndTime)
    print("end : model training")  

In [ ]:
# validating model

rf_predict = rf.predict(x_test)
mse = mean_squared_error(y_test, rf_predict)
rmse = np.sqrt(mse) # mean_abolute_error()
r2 = r2_score(y_test, rf_predict)
print('MSE:', mse)
print('RMSE:',rmse)
print('R square:', r2)

In [ ]:
sns.set_style('white')
sns.set_context("talk")

# sns.scatterplot(test_y,y_score)
plt.xlabel("Original",size = 20)
plt.ylabel("Predictions",size = 20)
plt.scatter(y_test, rf_predict, s=5)
ident = [y_test.min(),y_test.max()]
plt.plot(ident,ident,'--', color='black',linewidth=2)

plt.show()

In [ ]:
joblib.dump(xgb, modeldir+'/RF_regressor.pkl')

# Ridge, Lasso

In [ ]:
from sklearn.linear_model import Lasso
lasso=Lasso(alpha=0.0004)

In [ ]:
from sklearn.linear_model import Ridge
ridge=Ridge(alpha=10)

In [ ]:
lasso.fit(x_train,y_train)

In [ ]:
ridge.fit(x_train,y_train)

In [ ]:
#검증(평가)

lasso_predict = lasso.predict(x_test)
lasso_mse = mean_squared_error(y_test, lasso_predict)
lasso_rmse = np.sqrt(lasso_mse) # mean_abolute_error()
lasso_r2 = r2_score(y_test, lasso_predict)
print('MSE:', lasso_mse)
print('RMSE:',lasso_rmse)
print('R square:', lasso_r2)

In [ ]:
#검증(평가)

ridge_predict = ridge.predict(x_test)
ridge_mse = mean_squared_error(y_test, ridge_predict)
ridge_rmse = np.sqrt(ridge_mse) # mean_abolute_error()
ridge_r2 = r2_score(y_test, ridge_predict)
print('MSE:', ridge_mse)
print('RMSE:',ridge_rmse)
print('R square:', ridge_r2)

In [ ]:
sns.set_style('white')
sns.set_context("talk")


# sns.scatterplot(test_y,y_score)
plt.xlabel("Original",size = 20)
plt.ylabel("Predictions",size = 20)
plt.scatter(y_test, lasso_predict, s=5)
ident = [y_test.min(),y_test.max()]
plt.plot(ident,ident,'--', color='black',linewidth=2)

plt.show()

In [ ]:
# save trained models

import joblib

joblib.dump(lasso, modeldir+'/lasso.pkl')
joblib.dump(ridge, modeldir+'/ridge.pkl')

# XGBoost

In [ ]:
from xgboost import XGBRegressor

xgb = XGBRegressor(max_depth=8, min_child_weight= 0.001, gamma=0.01, learning_rate=0.1)

In [ ]:
with K.tf.device('/GPU:0'):
    
    print("step : model training")
    StartTime = datetime.now()
    print("StartTime :", StartTime)
    
    xgb.fit(x_train,y_train)
    
    EndTime = datetime.now()
    print("EndTime :", EndTime)
    print("end : model training")  

In [ ]:
# prediction performance validation

xgb_predict = xgb.predict(x_test)
mse = mean_squared_error(y_test, xgb_predict)
rmse = np.sqrt(mse) # mean_abolute_error()
r2 = r2_score(y_test, xgb_predict)
print('MSE:', mse)
print('RMSE:',rmse)
print('R square:', r2)

In [ ]:
sns.set_style('white')
sns.set_context("talk")

plt.title("XGB prediction graph")
plt.xlabel("Original",size = 20)
plt.ylabel("Predictions",size = 20)
plt.scatter(y_test, xgb_predict, s=5)
ident = [y_test.min(),y_test.max()]
plt.plot(ident,ident,'--', color='black',linewidth=2)

plt.show()

In [ ]:
joblib.dump(xgb, modeldir+'/XGB.pkl')